[課題のURL](https://diver.diveintocode.jp/curriculums/1626)

# Sprint21 自然言語処理入門

In [1]:
# IMDBをカレントフォルダにダウンロード
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
!cat aclImdb/README

--2020-07-01 13:52:10--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
ai.stanford.edu (ai.stanford.edu) をDNSに問いあわせています... 171.64.68.10
ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 84125825 (80M) [application/x-gzip]
`aclImdb_v1.tar.gz.1' に保存中

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  3.12MB/s 時間 27s        

2020-07-01 13:52:38 (2.92 MB/s) - `aclImdb_v1.tar.gz.1' へ保存完了 [84125825/84125825]

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000

In [2]:
from sklearn.datasets import load_files
train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target
test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target
# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos']


In [3]:
print("x : {}".format(x_train[0]))

x : Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.


In [4]:
mini_dataset = \
  ["This movie is very good.",
  "This film is a good",
  "Very bad. Very, very bad."]

In [5]:
# 1-gram

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()
# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

,a,bad,film,good,is,movie,this,very
0,0,0,0,1,1,1,1,1
1,1,0,1,1,1,0,1,0
2,0,2,0,0,0,0,0,3


In [6]:
# 2-gram

# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a good,bad very,film is,is a,is very,movie is,this film,this movie,very bad,very good,very very
0,0,0,0,0,1,1,0,1,0,1,0
1,1,0,1,1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,2,0,1


# 【問題1】BoWのスクラッチ実装

以下の3文のBoWを求められるプログラムをscikit-learnを使わずに作成してください。  
1-gramと2-gramで計算してください。  

This movie is SOOOO funny!!!  
What a movie! I never  
best movie ever!!!!! this movie  

## 1-gram

In [7]:
data = \
  ["This movie is SOOOO funny!!!",
  "What a movie! I never",
  "best movie ever!!!!! this movie"]

In [8]:
import re
import numpy as np

from collections import Counter

In [9]:
# 文章から単語を抽出する
def get_word_n_gram(data, n=1):
    words_n_gram = []

    for i in data:
        # 文章を単語に分ける
        words = re.split(r'\s|\,|\.|\(|\)', i.lower())

        for i in range(len(words)):
            words[i] = re.sub("[^a-zA-Z]", "",words[i])

        if n==1:
            for i in words:
                words_n_gram.append(i)

        elif n==2:
            for i in range(len(words)-1):
                words_n_gram.append(words[i] + " "+ words[i+1])
    
    return words_n_gram

# 単語から特徴量を作成する
def get_feature_names_n_gram(data, n=1):
    words_n_gram= get_word_n_gram(data, n)
    counter = Counter(words_n_gram)
    feature_name = sorted(counter)

    return feature_name

def get_bow(data, list_feature, n):
    bow = np.zeros((len(data), len(list_feature)))
    
    for i in range(len(data)):
        words_n_gram = []
        
        words = re.split(r'\s|\,|\.|\(|\)', data[i].lower())
        for l in range(len(words)):
            words[l] = re.sub("[^a-zA-Z]", "",words[l])
            
        if n==1:
            words_n_gram = words
            
        elif n==2:
            for m in range(len(words)-1):
                words_n_gram.append(words[m] + " "+ words[m+1])
  
        #print(words_n_gram)
        #print(list_feature)
        
        for j in range(len(words_n_gram)):
            for k in range(len(list_feature)):
                #print(words_n_gram[j], list_feature[k])
                
                if list_feature[k] == words_n_gram[j]:
                    #print(i, k)
                    bow[i][k] = bow[i][k]+1
    
    return bow

In [10]:
list_feature = get_feature_names_n_gram(data, 1) # 特徴量の一覧
bow = get_bow(data, list_feature, 1)

# DataFrameにまとめる
df = pd.DataFrame(bow, columns=list_feature, dtype = "int")
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0,0,0,1,0,1,1,0,1,1,0
1,1,0,0,0,1,0,1,1,0,0,1
2,0,1,1,0,0,0,2,0,0,1,0


In [11]:
# 参考(sklearn)

vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(data)).toarray()
# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0,0,0,1,0,1,1,0,1,1,0
1,1,0,0,0,1,0,1,1,0,0,1
2,0,1,1,0,0,0,2,0,0,1,0


## 2-gram

In [12]:
list_feature = get_feature_names_n_gram(data, 2) # 特徴量の一覧
bow = get_bow(data, list_feature, 2)

# DataFrameにまとめる
df = pd.DataFrame(bow, columns=list_feature, dtype = "int")
display(df)

,a movie,best movie,ever this,i never,is soooo,movie ever,movie i,movie is,soooo funny,this movie,what a
0,0,0,0,0,1,0,0,1,1,1,0
1,1,0,0,1,0,0,1,0,0,0,1
2,0,1,1,0,0,1,0,0,0,1,0


In [13]:
# 参考(sklearn)
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(data)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a movie,best movie,ever this,i never,is soooo,movie ever,movie i,movie is,soooo funny,this movie,what a
0,0,0,0,0,1,0,0,1,1,1,0
1,1,0,0,1,0,0,1,0,0,0,1
2,0,1,1,0,0,1,0,0,0,1,0


# 【問題2】TF-IDFの計算

In [14]:
import nltk
stop_words = nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/mori/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
len(stop_words)

179

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000)
bow = vectorizer.fit_transform(x_train)

In [18]:
df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names())
display(df)

,00,000,10,100,11,12,13,13th,14,15,...,yet,york,young,younger,youth,zero,zizek,zombie,zombies,zone
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.164505,0.0,0.0,0.136932,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.127480,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.085006,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.104143,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.000000,0.052665,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.096481,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
24996,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.074629,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.069721,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
24998,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.095550,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## 【問題3】TF-IDFを用いた学習

問題2で求めたベクトルを用いてIMDB映画レビューデータセットの学習・推定を行なってください。  
モデルは2値分類が行える任意のものを利用してください。  


ここでは精度の高さは求めませんが、最大の語彙数やストップワード、n-gramの数を変化させて影響を検証してみてください。  

1-gram 0.5566  
2-gram 0.56448  
最大の語彙数を500 0.6032  
最大の語彙数を250 0.73956  
最大の語彙数を100 0.56232  
→最大の語彙数の影響を受けやすい?  グリッドサーチなどで調べて方が良いかも  

In [19]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
lr = LogisticRegression()
lr.fit(bow.toarray(), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
bow_test = vectorizer.fit_transform(x_test)

In [22]:
y_pred = lr.predict(bow_test)
print("Accuracy（正解率）", accuracy_score(y_test, y_pred))

Accuracy（正解率） 0.5566


## 2-gramした場合

In [23]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000, ngram_range=(2, 2))
bow = vectorizer.fit_transform(x_train)

In [24]:
lr.fit(bow.toarray(), y_train)
bow_test = vectorizer.fit_transform(x_test)
y_pred = lr.predict(bow_test)
print("Accuracy（正解率）", accuracy_score(y_test, y_pred))

Accuracy（正解率） 0.56448


## 最大の語彙数を500に変更した場合

In [25]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=500, ngram_range=(1, 1))
bow = vectorizer.fit_transform(x_train)
lr.fit(bow.toarray(), y_train)
bow_test = vectorizer.fit_transform(x_test)
y_pred = lr.predict(bow_test)
print("Accuracy（正解率）", accuracy_score(y_test, y_pred))

Accuracy（正解率） 0.6032


## 最大の語彙数を250に変更した場合

In [26]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=250, ngram_range=(1, 1))
bow = vectorizer.fit_transform(x_train)
lr.fit(bow.toarray(), y_train)
bow_test = vectorizer.fit_transform(x_test)
y_pred = lr.predict(bow_test)
print("Accuracy（正解率）", accuracy_score(y_test, y_pred))

Accuracy（正解率） 0.73956


## 最大の語彙数を100に変更した場合

In [27]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=100, ngram_range=(1, 1))
bow = vectorizer.fit_transform(x_train)
lr.fit(bow.toarray(), y_train)
bow_test = vectorizer.fit_transform(x_test)
y_pred = lr.predict(bow_test)
print("Accuracy（正解率）", accuracy_score(y_test, y_pred))

Accuracy（正解率） 0.56232


# 【問題4】TF-IDFのスクラッチ実装

以下の3文のTF-IDFを求められるプログラムをscikit-learnを使わずに作成してください。  
標準的な式と、scikit-learnの採用している式の2種類を作成してください。正規化は不要です。  

In [28]:
data = [
  "This movie is SOOOO funny!!!",
  "What a movie! I never",
  "best movie ever!!!!! this movie"
]

## 標準的なTF-IDFの式

In [29]:
list_feature = get_feature_names_n_gram(data, 1) # 特徴量の一覧
bow = get_bow(data, list_feature, 1)

# 01, Term Frequencyを求める
# (サンプルd内のトークンtの出現回数（BoWと同じ)) /  サンプルdの全トークンの出現回数の和
tf =  bow/bow.sum()

# 02, Inverse Document Frequency:を求める
# log(サンプル数 / トークンtが出現するサンプル数)

# トークンtが出現するサンプル数を求める
bow_index = np.zeros(bow.shape[1])

for i in range(bow.shape[1]):
    bow_index[i] = np.count_nonzero(bow[:, i] > 0)

idf = np.log(len(data)/bow_index)

# 03, TF-IDFを求める
tf_ifd = tf*idf

df = pd.DataFrame(tf_ifd, columns=list_feature)
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0.000000,0.000000,0.000000,0.073241,0.000000,0.073241,0.0,0.000000,0.073241,0.027031,0.000000
1,0.073241,0.000000,0.000000,0.000000,0.073241,0.000000,0.0,0.073241,0.000000,0.000000,0.073241
2,0.000000,0.073241,0.073241,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.027031,0.000000


## scikit-learnのTF-IDF

In [30]:
list_feature = get_feature_names_n_gram(data, 1) # 特徴量の一覧
bow = get_bow(data, list_feature, 1)

# 01, Term Frequencyを求める
# (サンプルd内のトークンtの出現回数（BoWと同じ))
tf =  bow

# 02, Inverse Document Frequency:を求める
# log(サンプル数 / トークンtが出現するサンプル数)

# トークンtが出現するサンプル数を求める
bow_index = np.zeros(bow.shape[1])

for i in range(bow.shape[1]):
    bow_index[i] = np.count_nonzero(bow[:, i] > 0)

idf = np.log((1+len(data))/(1+bow_index)) + 1

# 03, TF-IDFを求める
tf_ifd = tf*idf

df = pd.DataFrame(tf_ifd, columns=list_feature)
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0.000000,0.000000,0.000000,1.693147,0.000000,1.693147,1.0,0.000000,1.693147,1.287682,0.000000
1,1.693147,0.000000,0.000000,0.000000,1.693147,0.000000,1.0,1.693147,0.000000,0.000000,1.693147
2,0.000000,1.693147,1.693147,0.000000,0.000000,0.000000,2.0,0.000000,0.000000,1.287682,0.000000


# 【問題5】コーパスの前処理

## Word2Vec

In [31]:
from gensim.models import Word2Vec
sentences = [['this', 'movie', 'is', 'very', 'good'], ['this', 'film', 'is', 'a', 'good'], ['very', 'bad', 'very', 'very', 'bad']]

model = Word2Vec(min_count=1, size=10) # 次元数を10に設定
model.build_vocab(sentences) # 準備
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter) # 学習

print("語彙の一覧 : {}".format(model.wv.vocab.keys()))

for vocab in model.wv.vocab.keys():
    print("{}のベクトル : \n{}".format(vocab, model.wv[vocab]))

語彙の一覧 : dict_keys(['this', 'movie', 'is', 'very', 'good', 'film', 'a', 'bad'])
thisのベクトル : 
[ 0.00564922  0.00073892 -0.02804589 -0.01707686  0.04455576  0.02809861
 -0.026614    0.02499997 -0.00336876 -0.00935752]
movieのベクトル : 
[-0.03369769 -0.03936495 -0.01846847  0.01826918  0.04780942 -0.00796487
 -0.014111   -0.02607367  0.01866732  0.02207876]
isのベクトル : 
[-0.02733105  0.00095104  0.03866376  0.01652475 -0.02622597  0.03155072
  0.02568799 -0.00025913  0.02161686 -0.01498434]
veryのベクトル : 
[ 0.00054233  0.03021604 -0.01822191  0.02902518  0.02173894  0.03131798
 -0.01711055 -0.02230836 -0.04903395  0.01237185]
goodのベクトル : 
[-0.01961618 -0.03343302  0.01696039  0.02275713 -0.01504859 -0.01760709
  0.04250591  0.01956521  0.02113725 -0.02527446]
filmのベクトル : 
[ 0.01662892 -0.00866692  0.03275421 -0.03432434 -0.02146358 -0.01397709
 -0.04618244 -0.02443717 -0.01044786  0.02071185]
aのベクトル : 
[-0.01284742  0.04613472  0.02710006 -0.04598175 -0.01981318  0.00891368
 -0.0210033  -0.0441762

/Users/mori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [32]:
model.wv.most_similar(positive="good", topn=3)

[('is', 0.565813422203064),
 ('bad', 0.03579186648130417),
 ('movie', 0.03367963060736656)]

In [33]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
vocabs = model.wv.vocab.keys()
tsne_model = TSNE(perplexity=40, n_components=2, init="pca", n_iter=5000, random_state=23)
vectors_tsne = tsne_model.fit_transform(model[vocabs])
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(vectors_tsne[:, 0], vectors_tsne[:, 1])
for i, word in enumerate(list(vocabs)):
    plt.annotate(word, xy=(vectors_tsne[i, 0], vectors_tsne[i, 1]))
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()

/Users/mori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


<Figure size 500x500 with 1 Axes>

## 前処理

コーパスの前処理として、特殊文字（!など）やURLの除去、大文字の小文字化といったことを行なってください。  
また、単語（トークン）はリストで分割してください。

In [34]:
test_data = [
    "Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.",
    'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clichés, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a doubt be you\'re choice over this one, even the "Turbulence"-movies are better. In fact, every other movie in the world is better than this one.',
    'Everyone plays their part pretty well in this "little nice movie". Belushi gets the chance to live part of his life differently, but ends up realizing that what he had was going to be just as good or maybe even better. The movie shows us that we ought to take advantage of the opportunities we have, not the ones we do not or cannot have. If U can get this movie on video for around $10, it´d be an investment!',
    'There are a lot of highly talented filmmakers/actors in Germany now. None of them are associated with this "movie".<br /><br />Why in the world do producers actually invest money in something like this this? You could have made 10 good films with the budget of this garbage! It\'s not entertaining to have seven grown men running around as dwarfs, pretending to be funny. What IS funny though is that the film\'s producer (who happens to be the oldest guy of the bunch) is playing the YOUNGEST dwarf.<br /><br />The film is filled with moments that scream for captions saying "You\'re supposed to laugh now!". It\'s hard to believe that this crap\'s supposed to be a comedy.<br /><br />Many people actually stood up and left the cinema 30 minutes into the movie. I should have done the same instead of wasting my time...<br /><br />Pain!'
]

In [35]:
def pre_process(data):
    
    sentences = []

    for i in data:
        # 文章を単語に分ける
        words = re.split(r'\s|\,|\.|\(|\)', i.lower())

        for j in range(len(words)):
            words[j] = re.sub("[^a-zA-Z]", '',words[j])
            words[j] = words[j].replace('br', '')   

        # 暫定対応 list内で文字数が0のものは抜く
        words = [i for i in words if len(i) != 0]

        sentences.append(words)
        
    return sentences

In [36]:
sentences = pre_process(test_data)
print(len(sentences))
sentences[0]

4


['zero',
 'day',
 'leads',
 'you',
 'to',
 'think',
 'even',
 'rethink',
 'why',
 'two',
 'boysyoung',
 'men',
 'would',
 'do',
 'what',
 'they',
 'did',
 'commit',
 'mutual',
 'suicide',
 'via',
 'slaughtering',
 'their',
 'classmates',
 'it',
 'captures',
 'what',
 'must',
 'be',
 'beyond',
 'a',
 'bizarre',
 'mode',
 'of',
 'being',
 'for',
 'two',
 'humans',
 'who',
 'have',
 'decided',
 'to',
 'withdraw',
 'from',
 'common',
 'civility',
 'in',
 'order',
 'to',
 'define',
 'their',
 'ownmutual',
 'world',
 'via',
 'coupled',
 'destruction',
 'it',
 'is',
 'not',
 'a',
 'perfect',
 'movie',
 'but',
 'given',
 'what',
 'moneytime',
 'the',
 'filmmaker',
 'and',
 'actors',
 'had',
 'it',
 'is',
 'a',
 'remarkable',
 'product',
 'in',
 'terms',
 'of',
 'explaining',
 'the',
 'motives',
 'and',
 'actions',
 'of',
 'the',
 'two',
 'young',
 'suicidemurderers',
 'it',
 'is',
 'better',
 'than',
 'elephant',
 'in',
 'terms',
 'of',
 'being',
 'a',
 'film',
 'that',
 'gets',
 'under',
 'ou

# 【問題6】Word2Vecの学習

Word2Vecの学習を行なってください。

In [37]:
len(x_train)

25000

In [38]:
sentences = pre_process(x_train)

model = Word2Vec(min_count=1, size=10) # 次元数を10に設定
model.build_vocab(sentences) # 準備
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter) # 学習

model.save("word2vec.gensim.model")

#print("語彙の一覧 : {}".format(model.wv.vocab.keys()))

#for vocab in model.wv.vocab.keys():
    #print("{}のベクトル : \n{}".format(vocab, model.wv[vocab]))

/Users/mori/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


# 【問題7】（アドバンス課題）ベクトルの可視化

得られたベクトルをt-SNEにより可視化してください。  
また、いくつかの単語を選びwv.most_similarを用いて似ている単語を調べてください。  

In [39]:
model.wv.most_similar(positive="happy", topn=5)

[('greatthis', 0.9387428760528564),
 ('yeah', 0.9178431630134583),
 ('right', 0.9029077291488647),
 ('afraid', 0.9013302326202393),
 ('bored', 0.9004983901977539)]

In [40]:
model.wv.most_similar(positive="movie", topn=5)

[('film', 0.9686204195022583),
 ('thing', 0.9535398483276367),
 ('word', 0.9470187425613403),
 ('sequel', 0.9302304983139038),
 ('mafiahitman', 0.9094391465187073)]

In [41]:
model.wv.most_similar(positive="reviews", topn=5)

[('comments', 0.9741317629814148),
 ('movies', 0.9570680260658264),
 ('sequels', 0.933691143989563),
 ('episodes', 0.9097708463668823),
 ('critics', 0.9082823395729065)]